In [ ]:
import numpy as np
from scipy.stats import poisson

MAX_BIKES = 20
MAX_MOVE = 5
RENTAL_REWARD = 10
MOVE_COST = 2
PARKING_COST = 4
DISCOUNT = 0.9

RENTAL_REQUEST_RATE = [3, 4]
RETURN_RATE = [3, 2]

def poisson_prob(n, lam):
    return poisson.pmf(n, lam)

def compute_expected_value(state, action, value_function, free_shuttle=False):
    total_value = 0
    total_reward = 0
    bikes_loc1 = min(MAX_BIKES, state[0] - action)
    bikes_loc2 = min(MAX_BIKES, state[1] + action)
    move_cost = MOVE_COST * abs(action)

    for rentals1 in range(0, MAX_BIKES + 1):
        for rentals2 in range(0, MAX_BIKES + 1):
            prob_rentals = poisson_prob(rentals1, RENTAL_REQUEST_RATE[0]) * poisson_prob(rentals2, RENTAL_REQUEST_RATE[1])
            actual_rentals1 = min(bikes_loc1, rentals1)
            actual_rentals2 = min(bikes_loc2, rentals2)
            reward = RENTAL_REWARD * (actual_rentals1 + actual_rentals2) - move_cost
            bikes_after_rentals1 = bikes_loc1 - actual_rentals1
            bikes_after_rentals2 = bikes_loc2 - actual_rentals2

            for returns1 in range(0, MAX_BIKES + 1):
                for returns2 in range(0, MAX_BIKES + 1):
                    prob_returns = poisson_prob(returns1, RETURN_RATE[0]) * poisson_prob(returns2, RETURN_RATE[1])
                    new_bikes_loc1 = min(MAX_BIKES, bikes_after_rentals1 + returns1)
                    new_bikes_loc2 = min(MAX_BIKES, bikes_after_rentals2 + returns2)
                    parking_cost = 0
                    if new_bikes_loc1 > 10:
                        parking_cost += PARKING_COST
                    if new_bikes_loc2 > 10:
                        parking_cost += PARKING_COST
                    total_reward = reward - parking_cost
                    prob = prob_rentals * prob_returns
                    total_value += prob * (total_reward + DISCOUNT * value_function[new_bikes_loc1, new_bikes_loc2])

    return total_value

def policy_iteration(free_shuttle=False):
    value_function = np.zeros((MAX_BIKES + 1, MAX_BIKES + 1))
    policy = np.zeros((MAX_BIKES + 1, MAX_BIKES + 1), dtype=int)
    stable = False

    while not stable:
        while True:
            delta = 0
            for i in range(MAX_BIKES + 1):
                for j in range(MAX_BIKES + 1):
                    state = (i, j)
                    old_value = value_function[i, j]
                    action = policy[i, j]
                    value_function[i, j] = compute_expected_value(state, action, value_function, free_shuttle)
                    delta = max(delta, abs(old_value - value_function[i, j]))
            if delta < 1e-4:
                break

        stable = True
        for i in range(MAX_BIKES + 1):
            for j in range(MAX_BIKES + 1):
                state = (i, j)
                old_action = policy[i, j]
                actions = range(-min(MAX_MOVE, state[0]), min(MAX_MOVE, state[1]) + 1)
                values = [compute_expected_value(state, action, value_function, free_shuttle) for action in actions]
                policy[i, j] = actions[np.argmax(values)]
                if old_action != policy[i, j]:
                    stable = False

    return policy, value_function

policy_original, value_original = policy_iteration(free_shuttle=False)

print("Policy (Original):")
print(policy_original)